In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import requests
import time
import urllib
import urllib.parse
import re
import csv
import string
import os.path
import sys
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:


scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)

TOKEN = open('key.txt', 'r').read().rstrip()
URL = "https://api.telegram.org/bot{}/".format(TOKEN)
triggers = {}
client = gspread.authorize(creds)
sheet = client.open('HenryBot commands').sheet1


def get_url(url):
    response = requests.get(url)
    content = response.content.decode("utf8")
    return content


def get_json_from_url(url):
    content = get_url(url)
    js = json.loads(content)
    return js


def get_updates(offset=None):
    url = URL + "getUpdates?timeout=100"
    if offset:
        url += "&offset={}".format(offset)
    js = False
    try:
        js = get_json_from_url(url)
    except Exception as e:
        print(e)
    return js


def get_last_update_id(updates):
    update_ids = []
    for update in updates["result"]:
        update_ids.append(int(update["update_id"]))
    return max(update_ids)


def get_last_chat_id_and_text(updates):
    num_updates = len(updates["result"])
    last_update = num_updates - 1
    text = updates["result"][last_update]["message"]["text"]
    chat_id = updates["result"][last_update]["message"]["chat"]["id"]
    return text, chat_id


def send_message(text, chat_id):
    text = urllib.parse.quote_plus(text)
    url = URL + "sendMessage?text={}&chat_id={}".format(text, chat_id)
    get_url(url)


def echo_all(updates):
    for update in updates["result"]:
        # try:
            text = update["message"]["text"]
            chat = update["message"]["chat"]["id"]
            response = respond(text).encode("utf-8")
            send_message(response, chat)
        # except Exception as e:
        #     print("Error in echo all: {}".format(e))


def respond(text):
    message = ""
    remove_punctuation = re.compile('[%s]' % re.escape(string.punctuation))
    command = text.split(' ', 1)[0]

    if command == '/start' or command == '/start@RU_Dad_bot':
        return "Hoi, ik ben Henry en je hebt me nu getriggerd. Je kan nieuwe trigger toevoegen met /add."

    if command == '/add' or command == '/add@RU_Dad_bot':
        try:
            value = text.split(' ', 1)[1]
            trigger, response = value.split(':', 1)
        except Exception as e:
            print("no keyword given\n{}".format(e))
            return "Add a new trigger by typing your trigger and response after /add, seperated by a colon (:)!"
        trigger = remove_punctuation.sub('', trigger.strip())
        # if trigger in triggers.keys():
        #     return "Wollah deze key bestaat al!\n"
        response = response.strip()
        add_triggers(trigger, response)
        return "Trigger toegevoegd!\n"

    # Respond to deletion of trigger words
    if command == '/delete' or command == '/delete@RU_Dad_bot':
        value = text.split(' ', 1)[1]
        del triggers[value]
        overwrite_remote_trigger_with_local()
        return '{} is verwijderd uit de triggers.'.format(value)

    if command == '/triggers' or command == '/triggers@RU_Dad_bot':
        try:
            value = text.split(' ', 1)[1]
            if value == "all":
                for key in triggers.keys():
                    message += "{}: {}\n".format(key, triggers[key])
                return message
        except Exception as e:
            pass
            # print("No second argument for trigger command given: {}".format(e))
        for key in triggers.keys():
            message += key + '\n'
        return message

    # Respond to 'I am'
    if re.search(r'ik ben \w+', text, re.I):
        matches = re.search(r'ik ben (\w+)', text, re.IGNORECASE)
        message += "Hoi {}, ik ben Henry Bot\n".format(matches.group(1))

    # Respond to added triggers
    text = remove_punctuation.sub('', text)
    print(text, triggers)
    for word in triggers.keys():
        regex = r'\b' + word + r'\b|\A' + word + r'\b '
        if re.search(regex, text, re.I):
            message += triggers[word]
    return message


def overwrite_remote_trigger_with_local():
    sheet.clear()
    sheet.append_row(['trigger', 'response'])
    for key, value in triggers.items():
        sheet.append_row([key, value])


def add_triggers(trigger, response):
    if trigger != '' and response != '':
        triggers[trigger] = response
        sheet.append_row([trigger, response])
    read_triggers()


def read_triggers():
    henry_commands = sheet.get_all_records()
    for trigger in henry_commands:
        triggers[trigger.get('trigger')] = trigger.get('response')


def main():
    read_triggers()
    last_update_id = None
    while True:
        try:
            updates = get_updates(last_update_id)
            if updates:
                if "result" in updates.keys():
                    if len(updates["result"]) > 0:
                        last_update_id = get_last_update_id(updates) + 1
                        echo_all(updates)
                else:
                    print("Bot was not found\n{}".format(updates))
            time.sleep(0.5)
        except Exception as e:
            print(e)

 #           os.execv(sys.executable, ['nohup', 'python3', 'main.py', '&'] + sys.argv)


if __name__ == '__main__':
    main()

In [2]:
#export
from telegram.ext import Updater, InlineQueryHandler, CommandHandler
import requests
import re
import lib

In [3]:
x = lib.DatabaseIO().read_data('triggers')
x.head()

,response
trigger,
Nick,Nick is niet bereikbaar tussen 9:00 en 17:00
kaas,Saak
Stijn,De Blockstijn
henry,Tot uw dienst!
probleem,https://www.youtube.com/watch?v=x9RsC_zZEfs


In [24]:
'\n'.join(x.index.tolist())


'Nick\nkaas\nStijn\nhenry\nprobleem\nherny\nmac\ntest\neens\nsecurity\nBauke'

In [27]:
henry_bot = HenryBot()

In [28]:
henry_bot.triggers.index.tolist()

['Nick',
 'kaas',
 'Stijn',
 'henry',
 'probleem',
 'herny',
 'mac',
 'test',
 'eens',
 'security',
 'Bauke']

In [28]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# This program is dedicated to the public domain under the CC0 license.
#
# THIS EXAMPLE HAS BEEN UPDATED TO WORK WITH THE BETA VERSION 12 OF PYTHON-TELEGRAM-BOT.
# If you're still using version 11.1.0, please see the examples at
# https://github.com/python-telegram-bot/python-telegram-bot/tree/v11.1.0/examples

"""
Simple Bot to reply to Telegram messages.

First, a few handler functions are defined. Then, those functions are passed to
the Dispatcher and registered at their respective places.
Then, the bot is started and runs until we press Ctrl-C on the command line.

Usage:
Basic Echobot example, repeats messages.
Press Ctrl-C on the command line or send a signal to the process to stop the
bot.
"""

import logging

from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
api_token = lib.config['telegram']['api_token']

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

class HenryBot:
    def __init__(self):
        self.db_conn = lib.DatabaseIO()
        self.triggers = x
#         self.triggers = self.db_conn.read_data('triggers')
    
    @staticmethod
    def start(update, context):
        """Send a message when the command /start is issued."""
        update.message.reply_text('Hoi, ik ben Henry en je hebt me nu getriggerd. Je kan nieuwe trigger toevoegen met /add.')
    
    def update_triggers(self):
        self.db_conn.write_data(self.triggers, 'triggers')
    
    def add(self, update, context):
        print(update.message.text)

    def delete(self, update, context):
        try:
            trigger = update.message.text.split(' ', 1)[1]
            self.triggers = self.triggers.drop(trigger)
            self.update_triggers()
        except IndexError:
            update.message.reply_text(f'Please add a trigger to remove. Format is /delete <trigger>')

    def get_triggers(self, update, context):
        message = ''
        for trigger, row in self.triggers.iterrows():
            message += f'{trigger}: {row.response}\n'
        update.message.reply_text(message)         

    def triggered(self, update, context):
        """Echo the user message."""
        update.message.reply_text(update.message.text)

    @staticmethod
    def help(update, context):
        """Send a message when the command /help is issued."""
        update.message.reply_text('Help!')


def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)


def main():
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    henry_bot = HenryBot()
    updater = Updater(api_token, use_context=True)

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # on different commands - answer in Telegram
    dp.add_handler(CommandHandler("start", henry_bot.start))
    dp.add_handler(CommandHandler("add", henry_bot.add))
    dp.add_handler(CommandHandler("delete", henry_bot.delete))
    dp.add_handler(CommandHandler("triggers", henry_bot.get_triggers))
    dp.add_handler(CommandHandler("help", henry_bot.help))

    # on noncommand i.e message - respond appropriately to the message on Telegram
    dp.add_handler(MessageHandler(Filters.text, henry_bot.triggered))

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()
    updater.idle()

In [29]:
main()

2019-05-19 14:15:03,378 - lib - INFO - Writing to triggers...
2019-05-19 14:15:09,285 - lib - INFO - Done.
2019-05-19 14:15:29,904 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...


In [6]:
import pandas as pd

In [7]:
x= pd.read_csv('commands.csv', sep=';')
x.head()

,trigger,response
0,Nick,Nick is niet bereikbaar tussen 9:00 en 17:00
1,kaas,Saak
2,Stijn,De Blockstijn
3,henry,Tot uw dienst!
4,probleem,https://www.youtube.com/watch?v=x9RsC_zZEfs


In [19]:
x

,response
trigger,
Nick,Nick is niet bereikbaar tussen 9:00 en 17:00
kaas,Saak
Stijn,De Blockstijn
henry,Tot uw dienst!
probleem,https://www.youtube.com/watch?v=x9RsC_zZEfs
herny,Are you herny?
mac,PC
test,beep boop
eens,oneens


In [22]:
lib.DatabaseIO().write_data(x, 'triggers')

In [40]:
#export
if __name__ == '__main__':
    main()

TypeError: __init__() got an unexpected keyword argument 'use_context'

In [ ]:
!python notebook2script.py bot.ipynb   